# Obtaining a RCCD Code using Sympy and Tchau-Spin

$\bullet$ Import Sympy packages

In [1]:
from sympy.physics.secondquant import (AntiSymmetricTensor, wicks,
        F, Fd, NO, evaluate_deltas, substitute_dummies, Commutator,
        simplify_index_permutations, PermutationOperator)
from sympy import (
    symbols, Rational, latex, Dummy
)

# For Sympy (simplification of equations)
pretty_dummies_dict = {
    'above': 'cdefgh',
    'below': 'klmno',
    'general': 'pqrstu'
}

$\bullet$ Define symbols for Sympy

In [2]:
i = symbols('i', below_fermi=True, cls=Dummy)
a = symbols('a', above_fermi=True, cls=Dummy)
j = symbols('j', below_fermi=True, cls=Dummy)
b = symbols('b', above_fermi=True, cls=Dummy)
p, q, r, s = symbols('p,q,r,s', cls=Dummy)

## Build the $\Phi$-normal ordered Hamiltonian

In [3]:
fock = AntiSymmetricTensor('f', (p,), (q,))
pr = NO(Fd(p)*F(q))
V = AntiSymmetricTensor('v',(p,q),(r,s))
pqsr = NO(Fd(p)*Fd(q)*F(s)*F(r))
H = fock*pr + Rational(1,4)*V*pqsr
H

AntiSymmetricTensor(f, (_p,), (_q,))*NO(CreateFermion(_p)*AnnihilateFermion(_q)) - AntiSymmetricTensor(v, (_p, _q), (_r, _s))*NO(CreateFermion(_p)*CreateFermion(_q)*AnnihilateFermion(_r)*AnnihilateFermion(_s))/4

## Build Cluster Operator

In [4]:
def get_T():
    i, j = symbols('i,j', below_fermi=True, cls=Dummy)
    a, b = symbols('a,b', above_fermi=True, cls=Dummy)
    t_abij = Rational(1,4)*AntiSymmetricTensor('t', (a,b), (i,j))*NO(Fd(a)*Fd(b)*F(j)*F(i))
    return t_abij

get_T()

-AntiSymmetricTensor(t, (_a, _b), (_i, _j))*NO(CreateFermion(_a)*CreateFermion(_b)*AnnihilateFermion(_i)*AnnihilateFermion(_j))/4

## Perform the Hausdorff expansion

<center> $\bar{H} = e^{-\hat{T}}\hat{H}_Ne^{\hat{T}}$ <br/>
    
 <p>&nbsp;</p>
    
<center>$\bar{H} = \hat{H}_N + [\hat{H}_N, \hat{T}] + \frac{1}{2}[[\hat{H}_N, \hat{T}], \hat{T}] + \frac{1}{3!}[[[\hat{H}_N, \hat{T}], \hat{T}], \hat{T}] + \frac{1}{4!}[[[[\hat{H}_N, \hat{T}], \hat{T}], \hat{T}], \hat{T}]$

In [5]:
C = Commutator
T = get_T()
print("commutator 1...")
comm1 = wicks(C(H, T))
comm1 = evaluate_deltas(comm1)
comm1 = substitute_dummies(comm1)

commutator 1...


In [6]:
T = get_T()
print("commutator 2...")
comm2 = wicks(C(comm1, T))
comm2 = evaluate_deltas(comm2)
comm2 = substitute_dummies(comm2)

commutator 2...


In [7]:
T = get_T()
print("commutator 3...")
comm3 = wicks(C(comm2, T))
comm3 = evaluate_deltas(comm3)
comm3 = substitute_dummies(comm3)

commutator 3...


In [8]:
T = get_T()
print("commutator 4...")
comm4 = wicks(C(comm3, T))
comm4 = evaluate_deltas(comm4)
comm4 = substitute_dummies(comm4)

commutator 4...


## Construct the Similarity Transformed Hamiltonian

In [9]:
eq = H + comm1 + comm2/2 + comm3/6 + comm4/24
eq = eq.expand()
eq = evaluate_deltas(eq)
eq = substitute_dummies(eq, new_indices=True,
        pretty_indices=pretty_dummies_dict)

## Get energy expression

<center>$E = \langle | \bar{H} e^{\hat{T}}| \rangle_C$

In [10]:
i, j, k, l = symbols('i,j,k,l', below_fermi=True)
a, b, c, d = symbols('a,b,c,d', above_fermi=True)
print()
print("CC Energy:")
energy = wicks(eq, simplify_dummies=True,
        keep_only_fully_contracted=True)
latex(energy)


CC Energy:


'\\frac{t^{cd}_{kl} v^{kl}_{cd}}{4}'

## Get $T_2$ amplitude equation

<center>$E = \langle \Phi_{ij}^{ab}| \bar{H} e^{\hat{T}}| \rangle_C$

<p>&nbsp;</p>

<center> $E = \langle |i^\dagger j^\dagger b a \bar{H} e^{\hat{T}}| \rangle_C$

In [11]:
eqT2 = wicks(NO(Fd(i)*Fd(j)*F(b)*F(a))*eq, simplify_dummies=True, 
             keep_only_fully_contracted=True, simplify_kronecker_deltas=True)
latex(eqT2)

'f^{k}_{i} t^{ab}_{jk} - f^{k}_{j} t^{ab}_{ik} - f^{a}_{c} t^{bc}_{ij} + f^{b}_{c} t^{ac}_{ij} - \\frac{t^{cd}_{ik} t^{ab}_{jl} v^{kl}_{cd}}{2} + \\frac{t^{cd}_{ij} t^{ab}_{kl} v^{kl}_{cd}}{4} + \\frac{t^{cd}_{ij} v^{ab}_{cd}}{2} + \\frac{t^{cd}_{jk} t^{ab}_{il} v^{kl}_{cd}}{2} - \\frac{t^{ac}_{kl} t^{bd}_{ij} v^{kl}_{cd}}{2} + t^{ac}_{ik} t^{bd}_{jl} v^{kl}_{cd} + t^{ac}_{ik} v^{bk}_{jc} - \\frac{t^{ac}_{ij} t^{bd}_{kl} v^{kl}_{cd}}{2} - t^{ac}_{jk} t^{bd}_{il} v^{kl}_{cd} - t^{ac}_{jk} v^{bk}_{ic} + \\frac{t^{ab}_{kl} v^{kl}_{ij}}{2} - t^{bc}_{ik} v^{ak}_{jc} + t^{bc}_{jk} v^{ak}_{ic} + v^{ab}_{ij}'

# Using Tchau-Spin

In [12]:
from tchau_spin import *
# For restricted CC
Tensor.rhf = True

$\bullet$ Define indexes

In [13]:
i,j,a,b = Index.new('ijab', 'abab', 'hhpp')
k,l,c,d = Index.new('klcd', 'xxxx', 'hhpp')
index_key = {'i':i,
             'j':j,
             'a':a,
             'b':b,
             'k':k,
             'l':l,
             'c':c,
             'd':d
}

## Processing Energy expression

In [14]:
E = eqfromlatex(latex(energy), index_key)
platex(E)

<IPython.core.display.Math object>

$\bullet$ Simplify Equation

In [24]:
E = E.simplify()
E = E.adapt_space()
platex(E)

<IPython.core.display.Math object>

$\bullet$ Process equation. It is necessary to identify the external indexes (in this case none). Also it is optional to provide names. Let's do it without these options first

In [25]:
pE = process_eq(eq=E)
pE.write_einsums_out(print_out=True)

name -= np.einsum('lkcd, klcd -> ', T_OoVv, V_OOVV, optimize = 'optimal')
name += 2.0*np.einsum('klcd, klcd -> ', T_OoVv, V_OOVV, optimize = 'optimal')



$\bullet$ The output is more useful if we define some options

In [27]:
mytensors = {
    'T_OoVv' : 'T2',
    'V_OOOO' : 'Voooo',
    'V_OOOV' : 'Vooov',
    'V_OOVV' : 'Voovv',
    'V_OVOV' : 'Vovov',
    'V_OVVV' : 'Vovvv',
    'V_VVVV' : 'Vvvvv',
    'f_OO' : 'fock_OO',
    'f_VV' : 'fock_VV',
    'f_OV' : 'fock_OV'
    
}

pE = process_eq(eq=E, name = 'CC_energy', tensor_labels = mytensors)
pE.write_einsums_out(print_out=True)

CC_energy -= np.einsum('lkcd, klcd -> ', T2, Voovv, optimize = 'optimal')
CC_energy += 2.0*np.einsum('klcd, klcd -> ', T2, Voovv, optimize = 'optimal')



## Processing Amplitude equation

In [18]:
T2 = eqfromlatex(latex(eqT2), index_key)
platex(T2)

<IPython.core.display.Math object>

$\bullet$ Simplify

In [21]:
T2 = T2.simplify()
T2 = T2.adapt_space()
platex(T2)

<IPython.core.display.Math object>

In [22]:
pT2 = process_eq(T2,i,j,a,b, name = 'newT2', tensor_labels = mytensors)
pT2.write_einsums_out(print_out=True)

newT2 += Voovv
newT2 += np.einsum('cb, ijac -> ijab', fock_VV, T2, optimize = 'optimal')
newT2 -= np.einsum('ik, kjab -> ijab', fock_OO, T2, optimize = 'optimal')
newT2 -= np.einsum('jk, ikab -> ijab', fock_OO, T2, optimize = 'optimal')
newT2 += np.einsum('ca, ijcb -> ijab', fock_VV, T2, optimize = 'optimal')
newT2 += np.einsum('ijcd, cdab -> ijab', T2, Vvvvv, optimize = 'optimal')
newT2 += 2.0*np.einsum('ikac, kjcb -> ijab', T2, Voovv, optimize = 'optimal')
newT2 -= np.einsum('kjac, ickb -> ijab', T2, Vovov, optimize = 'optimal')
newT2 -= np.einsum('kjbc, kica -> ijab', T2, Voovv, optimize = 'optimal')
newT2 += np.einsum('jkbc, kica -> ijab', T2, Voovv, optimize = 'optimal')
newT2 += np.einsum('kjcb, kica -> ijab', T2, Voovv, optimize = 'optimal')
newT2 -= np.einsum('kjcb, icka -> ijab', T2, Vovov, optimize = 'optimal')
newT2 += np.einsum('klab, ijkl -> ijab', T2, Voooo, optimize = 'optimal')
newT2 -= np.einsum('ikac, jckb -> ijab', T2, Vovov, optimize = 'optimal')
newT2 -= np.einsum(

## The equations above can be coded into a CC program. See the RCCD.py file for a complete code